In [1]:
from bs4 import BeautifulSoup
import requests
from requests import get
from urllib.parse import urljoin
import shutil
import pathlib

from typing import Dict, List

In [2]:
class Scraper:
    def __init__(self):
        self._session = None
        self.daten = {}

    def _materialien_login(self) -> None:
        login_url = ("https://www.wim.uni-mannheim.de"
                    "/schlather/teaching/aktuelle-semester/")
        soup = BeautifulSoup(get(login_url).text, "lxml")
        # Es gibt zwei Loginforms auf der Seite, wir benötigen die untere:
        login_form = soup.find(lambda tag: tag.name == "form" and tag.fieldset is not None)
        # Erstelle session objekt für Login
        self._session = requests.session()
        # Logindaten
        form_data = {
            'user': 'stochastik',
            'pass': 'schlather',
            'submit': 'Anmelden',
            'logintype': 'login',
            'pid': 0,
            'redirect_url': '/schlather/teaching/aktuelle-semester/materialien/',
            'tx_felogin_pi1[noredirect]': 0
        }
        # Login
        self._session.post(login_url, data=form_data)

    def _scrape_pdfs(self):
        base_url = ("https://www.wim.uni-mannheim.de"
                   "/schlather/teaching/aktuelle-semester/materialien/")
        soup = BeautifulSoup(self._session.get(base_url).text, "lxml")
        # Jede Vorlesung ist in einer 'gridelement accordion-item anchor' CSS Klasse:
        for vl in soup.find_all(class_="gridelement accordion-item anchor"):
            vl_name = vl.a.h4.text.replace("\xad", "")
            # Finde alle .pdf Dateien: (alternativ: href=re.compile(r"pdf$"))
            pdf_files = [ ]
            for link in vl.find_all("a", href=lambda h: h is not None and h.endswith("pdf")):
                pdf_files.append(urljoin(base_url, link["href"]))
            # Füge die Liste der .pdf-Links in das dict ein:
            self.daten[vl_name] = pdf_files
    
    def _download_file(self, url: str, pfad: pathlib.Path, filename: str) -> None:
        req = self._session.get(url, stream=True)
        if req.status_code == 200:
            with open(pfad / filename, "wb") as f:
                shutil.copyfileobj(req.raw, f)

    def download_all_files(self, verbose=False):
        self._materialien_login()
        self._scrape_pdfs()

        # Lade jetzt die Dateien je Vorlesung
        for vl_name, links in self.daten.items():
            # Erstelle Ordner für VL, falls dieser noch nicht existiert
            p = pathlib.Path().cwd() / vl_name.replace(":", "")
            if not p.exists():
                p.mkdir()
            # Lade Dateien runter
            for link in links:
                dateiname = link.split("/")[-1]
                if verbose:
                    print(f"Lade [{vl_name}]: {dateiname}...")
                self._download_file(link, p, dateiname)


In [3]:
s = Scraper()

In [5]:
s.download_all_files(verbose=True)

Lade [Aktuar (DAV)]: DAV_Uebersichtsplan_2022.pdf...
Lade [Vorstellungsrunden für BSc (Herbst) und MSc (Frühjahr)]: Vorstellung_BA_HS20.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: vortrag.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: FortR1.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: FortShiny.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: FortR2.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: FortR3.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: FortR4.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: FortR5.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: FortR6.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: FortR7.pdf...
Lade [HWS 21: Fortgeschrittenenkurs R]: FortR8.pdf...
